In [1]:
%load_ext autoreload
%autoreload 2

import argparse
import os
import math
import yaml
import logging
import random
import numpy as np
import sys
import imageio

import torch

sys.path.append("/scratch/ondemand28/harryscz/diffusion")

In [2]:
def parse_args(arg_list=None):
    parser = argparse.ArgumentParser(
        description="Unconditioned Video Diffusion Inference"
    )
    parser.add_argument(
        "--dataset-path", type=str, required=True,
        help="Directory containing input reference videos."
    )
    parser.add_argument(
        "--pretrained-model-name-or-path", type=str, required=True,
        help="Path or HF ID where transformer/vae/scheduler are stored."
    )
    parser.add_argument(
        "--checkpoint-path", type=str, required=True,
        help="Path to fine‐tuned checkpoint containing transformer state_dict."
    )
    parser.add_argument(
        "--output-dir", type=str, required=True,
        help="Where to write generated videos."
    )
    parser.add_argument(
        "--model-config", type=str, required=True,
        help="YAML file describing model params (height, width, num_reference, num_target, etc.)"
    )
    parser.add_argument(
        "--batch-size", type=int, default=1,
        help="Batch size per device (usually 1 for inference)."
    )
    parser.add_argument(
        "--num-inference-steps", type=int, default=50,
        help="Number of reverse diffusion steps to run."
    )
    parser.add_argument(
        "--mixed-precision", type=str, default="bf16",
        help="Whether to run backbone in 'fp16', 'bf16', or 'fp32'."
    )
    parser.add_argument(
        "--seed", type=int, default=42,
        help="Random seed for reproducibility."
    )
    parser.add_argument(
        "--shuffle", type=int, default=False,
        help="Whether to shuffle dataset. Usually False for inference."
    )
    parser.add_argument(
        "--is-uncond", type=bool, default=False,
        help=""
    )

    # If arg_list is None, argparse picks up sys.argv; 
    # otherwise it treats arg_list as the full argv list.
    return parser.parse_args(arg_list)

args = [
    "--dataset-path", "/scratch/ondemand28/harryscz/head_audio/data/data256/uv",
    "--pretrained-model-name-or-path", "/scratch/ondemand28/harryscz/model/CogVideoX-2b",
    "--checkpoint-path",  "/scratch/ondemand28/harryscz/head_audio/trainOutput/checkpoint-1000.pt",
    "--output-dir",  "/scratch/ondemand28/harryscz/diffusion/videoOut",
    "--model-config",  "/scratch/ondemand28/harryscz/diffusion/model_config.yaml",
    "--batch-size",  "1",
    "--num-inference-steps",  "50",
    "--mixed-precision",  "no",
    "--seed",  "42",
    "--shuffle",  "0",
]

args = parse_args(args)

with open(args.model_config, "r") as f: model_config = yaml.safe_load(f)


In [3]:
from accelerate import Accelerator
from accelerate.utils import DistributedDataParallelKwargs, ProjectConfiguration, set_seed
from accelerate.logging import get_logger

with open(args.model_config, "r") as f: model_config = yaml.safe_load(f)
if args.mixed_precision.lower() == "fp16":
    dtype = torch.float16
elif args.mixed_precision.lower() == "bf16":
    dtype = torch.bfloat16
else:
    dtype = torch.float32

accelerator_project_config = ProjectConfiguration(project_dir=args.output_dir,
                                                    logging_dir=os.path.join(args.output_dir, "logs"))
ddp_kwargs = DistributedDataParallelKwargs(find_unused_parameters=False)
accelerator = Accelerator(mixed_precision=args.mixed_precision,
                            project_config=accelerator_project_config,
                            kwargs_handlers=[ddp_kwargs])

# 2.4 Set random seed
if args.seed is not None:
    set_seed(args.seed + accelerator.process_index)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

logger = get_logger(__name__)
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s",
    datefmt="%m/%d/%Y %H:%M:%S",
    level=logging.INFO,
)
logger.info("Accelerator state:", accelerator.state)

--- Logging error ---
Traceback (most recent call last):
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/logging/__init__.py", line 1083, in emit
    msg = self.format(record)
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/logging/__init__.py", line 927, in format
    return fmt.format(record)
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/logging/__init__.py", line 663, in format
    record.message = record.getMessage()
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/logging/__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/lib/python3.9/runpy.py", line 87, in

In [4]:
#### Dataset #####
# Video data have shape [B, C, F, H, W]

from data.CustomDataset import VideoDataset 
from torch.utils.data import DataLoader, DistributedSampler

dataset = VideoDataset(
    videos_dir=args.dataset_path,
    num_ref_frames=1,
    num_target_frames=49
)
if args.shuffle:
    sampler = DistributedSampler(
        dataset,
        num_replicas=accelerator.num_processes,
        rank=accelerator.process_index,
        shuffle=True
    )
else:
    sampler = None
data_loader = DataLoader(
    dataset,
    batch_size=args.batch_size,
    # sampler=sampler,
    collate_fn=lambda x: x[0],   # since dataset returns already‐batched items
    num_workers=2,
    pin_memory=True,
)
logger.info(f"Number of test examples: {len(data_loader)}")

06/07/2025 18:55:06 - INFO - __main__ - Number of test examples: 10


In [5]:
#### Load Model ####
device = "cuda"
dtype = torch.float32

from diffusers import AutoencoderKLCogVideoX, CogVideoXDPMScheduler
from cap_transformer import CAPVideoXTransformer3DModel

transformer = CAPVideoXTransformer3DModel.from_pretrained(
    args.pretrained_model_name_or_path,
    low_cpu_mem_usage=False,
    device_map=None,
    ignore_mismatched_sizes=True,
    subfolder="transformer",
    torch_dtype=torch.float32,
    cond_in_channels=1,  # only one channel (the ref_mask)
    sample_width=model_config["width"] // 8,
    sample_height=model_config["height"] // 8,
    max_text_seq_length=1,
    max_n_references=model_config["max_n_references"],
    apply_attention_scaling=model_config["use_growth_scaling"],
    use_rotary_positional_embeddings=False,
)

vae = AutoencoderKLCogVideoX.from_pretrained(
    args.pretrained_model_name_or_path, subfolder="vae"
)
scheduler = CogVideoXDPMScheduler.from_pretrained(
    args.pretrained_model_name_or_path, subfolder="scheduler",
)

vae.eval().to(dtype)
transformer.eval().to(dtype)

vae, transformer, scheduler, data_loader = accelerator.prepare(vae, transformer, scheduler, data_loader)

Some weights of the model checkpoint at /scratch/ondemand28/harryscz/model/CogVideoX-2b were not used when initializing CAPVideoXTransformer3DModel: 
 ['patch_embed.text_proj.weight, patch_embed.text_proj.bias']
Some weights of CAPVideoXTransformer3DModel were not initialized from the model checkpoint at /scratch/ondemand28/harryscz/model/CogVideoX-2b and are newly initialized: ['patch_embed.cond_proj.bias', 'patch_embed.audio_proj.weight', 'patch_embed.audio_proj.bias', 'patch_embed.cond_proj.weight', 'patch_embed.ref_temp_proj.bias', 'patch_embed.ref_temp_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# ckpt = torch.load(args.checkpoint_path, map_location="cpu")
# if "state_dict" in ckpt:
#     raw_state_dict = ckpt["state_dict"]
# elif "model_state_dict" in ckpt:
#     raw_state_dict = ckpt["model_state_dict"]
# else:
#     # If the .pt is literally just a pure state_dict, do this:
#     raw_state_dict = ckpt

# clean_state_dict = {}
# for key, val in raw_state_dict.items():
#     new_key = key
#     # e.g. if your keys start with "module.", remove it:
#     if key.startswith("module."):
#         new_key = key[len("module."):]
#     # or if saved under "model.", do:
#     # if key.startswith("model."):
#     #     new_key = key[len("model."):]
#     clean_state_dict[new_key] = val
# missing, unexpected = transformer.load_state_dict(clean_state_dict, strict=False)

# print("==> Missing keys (these will be randomly initialized because they weren't in the checkpoint):")
# for k in missing:
#     print("   ", k)
# print("==> Unexpected keys (these were in the checkpoint but didn't match any parameter in your model):")
# for k in unexpected:
#     print("   ", k)

In [7]:
for batch_id, batch in enumerate(data_loader):
    print(batch.keys())
    for chunk_id in range(len(batch["video_chunks"])):
        print(batch["video_chunks"][chunk_id].shape)
        print(batch["cond_chunks"].keys())  # Mask for each frames over H and W and channel suggesting which one works as a condition
                                            # list of tensor masks for cond chunks
        print(batch['cond_chunks']['ref_mask'][0].shape)
        print(batch["chunk_is_ref"]) # list of length frame of bool saying which on is a condition 
        print(batch["raw_audio"]) # passed as none
    break

dict_keys(['video_chunks', 'cond_chunks', 'chunk_is_ref', 'raw_audio'])
torch.Size([1, 3, 50, 256, 256])
dict_keys(['ref_mask'])
torch.Size([1, 50, 256, 256, 3])
[tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False, False],
       device='cuda:0')]
None


In [93]:
class BatchVideoPipeline(DiffusionPipeline):
    """
    A custom diffusion pipeline that mirrors your manual inference loop,
    but inherits from DiffusionPipeline to leverage no-grad, mixed-precision,
    and buffer reuse for maximum efficiency.
    """
    def __init__(
        self,
        vae,
        transformer,
        scheduler,
    ):
        super().__init__()
        self.register_modules(vae=vae, transformer=transformer, scheduler=scheduler)

        # Scale factors for spatial/temporal axes
        self.vae_scale_factor_spatial = 2 ** (len(self.vae.config.block_out_channels) - 1)
        self.vae_scale_factor_temporal = getattr(self.vae.config, "temporal_compression_ratio", 1)

        # Video post-processor
        self.video_processor = VideoProcessor(vae_scale_factor=self.vae_scale_factor_spatial)

    @torch.no_grad()
    def __call__(
        self,
        batch: Dict[str, Union[List[torch.FloatTensor], Dict[str, List[torch.FloatTensor]]]],
        num_inference_steps: int = 50,
        guidance_scale: float = 1.0,
        generator: Optional[Union[torch.Generator, List[torch.Generator]]] = None,
        output_type: str = "pil",
        return_dict: bool = False
    ) -> Union[List, Dict]:
        device = self._execution_device
        dtype = self.transformer.dtype

        # 1) Extract & encode
        latent_chunks: List[torch.Tensor] = []
        ref_mask_chunks: List[torch.Tensor] = []
        sequence_infos: List[tuple] = []

        for i, video in enumerate(batch["video_chunks"]):
            # video: [B, C, F, H, W]
            video = video.to(device=device, dtype=dtype)
            dist = self.vae.encode(video).latent_dist.sample()
            latent = dist * self.vae.config.scaling_factor
            latent = latent.permute(0, 2, 1, 3, 4).contiguous()  # [B, F, C_z, h, w]
            latent_chunks.append(latent)

            # mask: batch["cond_chunks"]["ref_mask"][i] shape [B, F, H, W, C_mask]
            rm = batch["cond_chunks"]["ref_mask"][i]
            rm = rm.to(device=device, dtype=dtype).permute(0, 4, 1, 2, 3)
            # now [B, C_mask, F, H, W]
            ref_mask_chunks.append(rm)

            # sequence info
            is_ref = batch.get("chunk_is_ref", [False] * len(latent_chunks))[i]
            seq = torch.arange(0, latent.shape[1], device=device)
            sequence_infos.append((is_ref, seq))

        # 2) Build 2× for classifier-free guidance
        latents = latent_chunks
        masks   = [torch.cat([m, torch.zeros_like(m)], dim=0) for m in ref_mask_chunks]
        # keep ref_latents for mixing
        ref_latents = [torch.cat([z, torch.zeros_like(z)], dim=0) for z in latent_chunks]

        # 3) dummy audio/text embeddings (adjust if you have real ones)
        B2 = latents[0].shape[0] * 2
        total_F = sum(z.shape[1] for z in latents)
        audio_embeds = torch.zeros((B2, total_F, 768), dtype=dtype, device=device)
        text_embeds  = torch.zeros((B2, 1,
            self.transformer.config.attention_head_dim * self.transformer.config.num_attention_heads
        ), dtype=dtype, device=device)

        # 4) timesteps
        timesteps, _ = retrieve_timesteps(self.scheduler, num_inference_steps, device=device)

        # 5) optional fuse QKV once
        # try:
        #     self.transformer.fuse_qkv_projections()
        # except Exception:
        #     pass

        # 6) denoising loop
        old_pred_original_samples = [None] * len(latents)
        for i, t in enumerate(timesteps):
            latent_model_inputs = [torch.cat([chunks] * 2, dim=0)for chunks in latents]
            B2, F, C, H, W = latent_model_inputs[0].shape
            # one zero condition tensor
            zero_cond = torch.zeros((B2, F, 1, H, W), dtype=dtype, device=device)

            # single forward
            noise_preds = self.transformer(
                hidden_states=latent_model_inputs,
                encoder_hidden_states=text_embeds,
                audio_embeds=audio_embeds,
                condition=[zero_cond] * len(latent_model_inputs),
                sequence_infos=[[False, torch.arange(chunk.shape[1])]for chunk in latents],
                timestep=t.expand(B2),
                image_rotary_emb=None,
                return_dict=False,
            )[0]

            print(latent_model_inputs[0].shape)
            print(noise_preds[0].shape)

            # apply guidance, scheduler.step, then mixing
            new_latents = []
            new_old_pred_original_samples = []

            for noise_pred, old_pred_original_sample, latent in zip(noise_preds, old_pred_original_samples, latent_chunks):
                print(old_pred_original_samples)
                noise_pred, noise_pred_uncond = noise_pred.chunk(2, dim=0)
                noise_pred = noise_pred_uncond + guidance_scale * (noise_pred - noise_pred_uncond)
            
                latent, old_pred_original_sample = scheduler.step(
                    noise_pred,
                    old_pred_original_sample,
                    t,
                    timesteps[i - 1] if i > 0 else None,
                    latent,
                    eta=0.0,
                    generator=generator
                )

                new_latents.append(latent)
                new_old_pred_original_samples.append(old_pred_original_sample)

            latents = new_latents
            old_pred_original_samples = list(new_old_pred_original_samples)

        # 7) decode to videos
        videos = []
        for latent in latents:
            dec = latent.permute(0, 2, 1, 3, 4) / self.vae.config.scaling_factor
            frames = self.vae.decode(dec).sample
            video = self.video_processor.postprocess_video(video=frames, output_type=output_type)
            videos.append(video)

        return {"frames": videos} if return_dict else videos

In [94]:
generator = torch.Generator(device=device)
generator.manual_seed(args.seed)
pipe = BatchVideoPipeline(vae, transformer, scheduler)
pipe = pipe.to(device).to(torch.float32)   
batch = next(iter(data_loader))

In [95]:
videos = pipe(batch, num_inference_steps=50, guidance_scale=1, generator=generator)

cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCog

torch.Size([2, 13, 16, 32, 32])
torch.Size([2, 13, 16, 32, 32])
[None]


cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCog

torch.Size([2, 13, 16, 32, 32])
torch.Size([2, 13, 16, 32, 32])


cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCog

...

torch.Size([2, 13, 16, 32, 32])
torch.Size([2, 13, 16, 32, 32])


cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.
cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCog

[tensor([[[[[ 2.2905,  2.1421,  2.2080,  ...,  2.1684,  2.1177,  2.1755],
           [ 1.7784,  1.9158,  1.6299,  ...,  1.9094,  1.5710,  1.9314],
           [ 2.2464,  2.1221,  2.1549,  ...,  2.1146,  2.0774,  2.0882],
           ...,
           [ 1.1976,  1.4203,  1.1696,  ...,  1.4435,  1.1398,  1.5134],
           [ 1.6395,  1.5931,  1.6437,  ...,  1.5532,  1.5631,  1.6312],
           [ 1.1925,  1.4439,  1.2021,  ...,  1.4182,  1.1483,  1.5097]],

          [[-2.4293, -1.5978, -2.1799,  ..., -1.5698, -2.0947, -2.2587],
           [-1.9514, -2.2483, -1.7398,  ..., -1.4791, -2.0876, -1.7362],
           [-1.8494, -2.0031, -1.5593,  ..., -2.1538, -1.7302, -2.2964],
           ...,
           [-1.9166, -1.7930, -2.2556,  ..., -1.5915, -1.9133, -1.2709],
           [-1.7256, -2.1079, -1.8349,  ..., -2.1780, -1.4851, -1.8188],
           [-1.3576, -1.7966, -2.0747,  ..., -1.5655, -1.6420, -1.3604]],

          [[ 1.3227,  1.9453,  1.1693,  ...,  0.9463,  1.8072,  0.7798],
           [ 1

cross_attention_kwargs ['scale_multiplier'] are not expected by FusedCogVideoXAttnProcessor2_0 and will be ignored.


torch.Size([2, 13, 16, 32, 32])
torch.Size([2, 13, 16, 32, 32])
[tensor([[[[[ 2.2229,  2.0711,  2.1377,  ...,  2.0983,  2.0446,  2.1056],
           [ 1.7994,  1.8826,  1.6461,  ...,  1.8760,  1.5853,  1.8987],
           [ 2.1774,  2.0505,  2.0829,  ...,  2.0427,  2.0030,  2.0155],
           ...,
           [ 1.2000,  1.3712,  1.1711,  ...,  1.3952,  1.1404,  1.4674],
           [ 1.5510,  1.5046,  1.5554,  ...,  1.4634,  1.4723,  1.5439],
           [ 1.1947,  1.3957,  1.2047,  ...,  1.3691,  1.1491,  1.4635]],

          [[-2.2903, -1.3585, -2.0329,  ..., -1.3297, -1.9450, -2.0405],
           [-1.7752, -2.1200, -1.5568,  ..., -1.3263, -1.9157, -1.5916],
           [-1.6919, -1.7769, -1.3924,  ..., -1.9323, -1.5688, -2.0795],
           ...,
           [-1.7393, -1.6503, -2.0891,  ..., -1.4423, -1.7359, -1.1115],
           [-1.5641, -1.8850, -1.6768,  ..., -1.9573, -1.3159, -1.5866],
           [-1.1624, -1.6539, -1.9024,  ..., -1.4155, -1.4559, -1.2038]],

          [[ 1.1217,  1

In [70]:
x = iter(data_loader)

In [71]:
for i in range(10):
    print(next(x)['video_chunks'][0].shape)

torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])
torch.Size([1, 3, 50, 256, 256])


In [ ]:
from diffusers.pipelines.stable_diffusion.pipeline_stable_diffusion import retrieve_timesteps
from tqdm.notebook import tqdm

latent_chunks        = [] # [B, F, C, H, W]
uncond_latent_chunks = []
cond_chunks          = []
uncond_chunks        = []
uncond_latents = []
ref_mask_chunks      = []
sequence_infos       = []
ref_latents = []

def encode_video(video):
    
    video = video.to(device, dtype=vae.dtype)
    # VAE takes shape of [B, C, F, H, W]
    with torch.no_grad():
        latent_dist = vae.encode(video).latent_dist.sample() * vae.config.scaling_factor

    # Permute back to [B, F, C, H, W]
    return latent_dist.permute(0, 2, 1, 3, 4).to(memory_format=torch.contiguous_format) 


for batch_id, batch in enumerate(data_loader):
    for chunk_id in range(len(batch["video_chunks"])):
        B, C, F, H, W = batch['video_chunks'][chunk_id].shape
        latent_chunk = encode_video(batch["video_chunks"][chunk_id]) # [B, C, F, H, W]
        latent_chunks.append(latent_chunk)
        uncond_latent_chunks.append(latent_chunk * 0.)
        cond_chunk = {key : batch['cond_chunks'][key][chunk_id] for key in batch['cond_chunks'].keys()}
        ref_mask_chunks.append(cond_chunk["ref_mask"].permute(0,1,4,2,3))
        uncond_chunks.append(cond_chunk["ref_mask"] * 0.)
        uncond_latents.append(latent_chunk * 0.)
        sequence_infos.append((False, torch.arange(0, latent_chunk.shape[1], device=device)))
        ref_latents.append(torch.cat([latent_chunk, uncond_latents[chunk_id]], dim=0))

num_inference_steps=50
generator = torch.Generator(device=device)
generator.manual_seed(args.seed)

timesteps, num_inference_steps = retrieve_timesteps(scheduler, num_inference_steps, device)

latent_channels = 16 # transformer.config.in_channels

T_total = sum(chunk.shape[1] for chunk in latent_chunks)  # = 4 * (num_chunks)
inner_dim = (
    accelerator.unwrap_model(transformer).config.num_attention_heads
    * accelerator.unwrap_model(transformer).config.attention_head_dim
)

text_embeds        = torch.zeros((B, 1, inner_dim), dtype=dtype, device=device)
text_in = torch.cat([text_embeds] * 2, dim=0)

audio_embeds   = torch.zeros((B, T_total, 768), dtype=dtype, device=device)
uncond_audio_embeds = audio_embeds * 0.0
audio_in = torch.cat([audio_embeds] * 2, dim=0)

initial_latents = []
ref_latents = []

for chunk_id in range(len(latent_chunks)):
    # Add Reference latent if used as refernce or else add random noise
    if sequence_infos[chunk_id][0]:
        noisy_latent = latent_chunks[chunk_id]
    else:
        noisy_latent = torch.randn(
            latent_chunks[chunk_id].shape,
            generator=generator,
            device=device,
            dtype=dtype
        )
    
    initial_latents.append(noisy_latent)

    ref_latents.append(latent_chunks[chunk_id])

    latents = initial_latents

num_warmup_steps = max(len(timesteps) - num_inference_steps * scheduler.order, 0)

guidance_scale =  1
# noise_preds = 0
for i, t in tqdm(enumerate(timesteps), desc="Denoising"):
    print(i)
    latent_model_inputs = [torch.cat([chunks] * 2, dim=0)for chunks in latent_chunks]
    old_pred_original_samples = [None] * len(latent_chunks)
    timestep = t.expand(latent_chunks[0].shape[0])
    current_sampling_percent = i / len(timesteps)
    B, F, C, H, W = latent_chunks[0].shape

    with torch.no_grad():
        noise_preds = transformer(
            hidden_states=latent_model_inputs,
            encoder_hidden_states=text_in,
            audio_embeds=audio_in,
            condition=[torch.zeros((B * 2, F, 1, H, W), device=device, dtype=dtype) for item in latent_model_inputs],
            sequence_infos=[[False, torch.arange(chunk.shape[1])]for chunk in latent_chunks],
            timestep=timestep,
            image_rotary_emb=None,
            return_dict=False,
            growth_factor=None,
        )[0]

    new_latents = []
    new_old_pred_original_samples = []

    for noise_pred, old_pred_original_sample, latent in zip(noise_preds, old_pred_original_samples, latent_chunks):
        noise_pred, noise_pred_uncond = noise_pred.chunk(2)
        noise_pred = noise_pred_uncond + guidance_scale * (noise_pred - noise_pred_uncond)
    
        latent, old_pred_original_sample = scheduler.step(
            noise_pred,
            old_pred_original_sample,
            t,
            timesteps[i - 1] if i > 0 else None,
            latent,
            eta=0.0,
            generator=generator
        )

        new_latents.append(latent)
        new_old_pred_original_samples.append(old_pred_original_sample)

    latents = new_latents
    old_pred_original_samples = new_old_pred_original_samples

[2025-06-07 18:53:09,021] [INFO] [real_accelerator.py:254:get_accelerator] Setting ds_accelerator to cuda (auto detect)


06/07/2025 18:53:09 - INFO - root - gcc -pthread -B /scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/include -I/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/include -fPIC -O2 -isystem /scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/include -fPIC -c /tmp/tmp7fn1wcuz/test.c -o /tmp/tmp7fn1wcuz/test.o


06/07/2025 18:53:09 - INFO - root - gcc -pthread -B /scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/compiler_compat /tmp/tmp7fn1wcuz/test.o -laio -o /tmp/tmp7fn1wcuz/a.out
/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
06/07/2025 18:53:10 - INFO - root - gcc -pthread -B /scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/compiler_compat -Wno-unused-result -Wsign-compare -DNDEBUG -O2 -Wall -fPIC -O2 -isystem /scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/include -I/scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/include -fPIC -O2 -isystem /scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/include -fPIC -c /tmp/tmpm8zzhwth/test.c -o /tmp/tmpm8zzhwth/test.o
06/07/2025 18:53:10 - INFO - root - gcc -pthread -B /scratch/ondemand28/harryscz/anaconda3/envs/pytorch3d/compiler_compat /tmp/tmpm8zzhwth/test.o -L/usr/local/cuda -L/usr/local/cuda/lib64 -lcufile -o

Denoising: 0it [00:00, ?it/s]

0
